In [1]:
!wget https://nlp.stanford.edu/sentiment/trainDevTestTrees_PTB.zip
!unzip trainDevTestTrees_PTB.zip

--2022-08-08 17:55:15--  https://nlp.stanford.edu/sentiment/trainDevTestTrees_PTB.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 789539 (771K) [application/zip]
Saving to: ‘trainDevTestTrees_PTB.zip.2’

trainDevTestTrees_P 100%[===================>] 771.03K   410KB/s    in 1.9s    

2022-08-08 17:55:18 (410 KB/s) - ‘trainDevTestTrees_PTB.zip.2’ saved [789539/789539]

Archive:  trainDevTestTrees_PTB.zip
replace trees/dev.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [2]:
import nltk
a = nltk.corpus.BracketParseCorpusReader("trees", "(train|dev|test)\.txt")

text = {}
labels = {}
keys = ['train', 'dev', 'test']
for k in keys :
    text[k] = [x.leaves() for x in a.parsed_sents(k+'.txt') if x.label() != '2']
    labels[k] = [int(x.label()) for x in a.parsed_sents(k+'.txt') if x.label() != '2']
    print(len(text[k]))
    
import spacy
nlp = spacy.load('en_core_web_sm', disable=['parser', 'tagger', 'ner'])
import re

def tokenize(text) :
    text = " ".join(text)
    text = text.replace("-LRB-", '')
    text = text.replace("-RRB-", " ")
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    tokens = " ".join([t.text.lower() for t in nlp(text)])
    return tokens

for k in keys :
    text[k] = [tokenize(t) for t in text[k]]
    labels[k] = [1 if x >= 3 else 0 for x in labels[k]]

6920
872
1821


In [4]:
import pandas as pd
df_texts = []
df_labels = []
df_exp_split = []

for k in keys :
    df_texts += text[k]
    df_labels += labels[k]
    df_exp_split += [k]*len(text[k])
    
data = pd.DataFrame({'text' : df_texts, 'label' : df_labels, 'exp_split' : df_exp_split}) 

In [5]:
import os
os.makedirs('data', exist_ok=True)

In [7]:
import spacy
nlp = spacy.load('en_core_web_sm')

def clean_text(text) :
    return "\n".join([" ".join([t.text.strip() for t in sent]) for sent in nlp(text).sents])

In [8]:
from tqdm import tqdm

tqdm.pandas()

data["text"] = data.text.progress_apply(lambda x : clean_text(x))
data["label_id"] = data.label.apply(lambda x : "positive" if x == 1 else "negative")


100%|██████████| 9613/9613 [00:36<00:00, 263.28it/s]


In [9]:
pd.options.mode.chained_assignment = None 
train = data.loc[data.exp_split == "train"]
train["annotation_id"] = list(range(len(train)))
train.annotation_id = train.annotation_id.apply(lambda x: "train_" + str(x))
train.to_csv("data/train.csv", index = False)

dev = data.loc[data.exp_split == "dev"]
dev["annotation_id"] = list(range(len(dev)))
dev.annotation_id = dev.annotation_id.apply(lambda x: "dev_" + str(x))
dev.to_csv("data/dev.csv", index = False)

test = data.loc[data.exp_split == "test"]
test["annotation_id"] = list(range(len(test)))
test.annotation_id = test.annotation_id.apply(lambda x: "test_" + str(x))
test.to_csv("data/test.csv", index = False)